# Topos - 2019 Data Eng Intern Assignment

### By Wei Tu (weitu@andrew.cmu.edu)
#### Date: May 28, 2019

In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import progressbar
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Part 1 - Collect data from List of United States cities by population wikipedia page

### Set and connect to the URL

In [2]:
# Set the URL to scrape from
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'

# Connect to the URL
response = requests.get(url)
response

<Response [200]>

### Parse HTML and save to BeautifulSoup object

In [3]:
# Parse HTML and save to BeautifulSoup object¶
soup = BeautifulSoup(response.text, "html.parser")

### Locate to the cities information table

In [4]:
# Get the cities information table
table = soup.find_all('table')[4]

### Locate to the data in each row 

In [5]:
# Get rows from the table, this includes the header
rows = table.find_all('tr')

### Clean and format each row's data

#### - Remove the HTML tag from each row

In [6]:
# Create a list to store rows of data after first cleaning
list_rows = []

# Clean up data in each row
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>|\[.*\]|,')
    clean2 = re.sub(clean, '',str_cells).strip('[]')
    list_rows.append(clean2)

# Review parts of the result    
list_rows[1:3]

['1\n New York City\n \xa0New York\n 8398748\n 8175133\n +2.74%\n 301.5\xa0sq\xa0mi\n 780.9\xa0km2\n 28317/sq\xa0mi\n 10933/km2\n 40°39′49″N 73°56′19″W\ufeff / \ufeff40.6635°N 73.9387°W\ufeff / 40.6635; -73.9387\ufeff (1 New York City)\n',
 '2\n Los Angeles\n \xa0California\n 3990456\n 3792621\n +5.22%\n 468.7\xa0sq\xa0mi\n 1213.9\xa0km2\n 8484/sq\xa0mi\n 3276/km2\n 34°01′10″N 118°24′39″W\ufeff / \ufeff34.0194°N 118.4108°W\ufeff / 34.0194; -118.4108\ufeff (2 Los Angeles)\n']

#### - Put into the dataframe for further data manipulation and cleaning 

In [7]:
df = pd.DataFrame(list_rows[1:])
df.head(5)

,0
0,1\n New York City\n New York\n 8398748\n 8175...
1,2\n Los Angeles\n California\n 3990456\n 3792...
2,3\n Chicago\n Illinois\n 2705994\n 2695598\n ...
3,4\n Houston\n Texas\n 2325502\n 2100263\n +10...
4,5\n Phoenix\n Arizona\n 1660272\n 1445632\n +...


In [8]:
# Split to columns
df1 = df[0].str.split('\\n', expand=True)
df1.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,New York City,New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,28317/sq mi,10933/km2,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W...,
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,1213.9 km2,8484/sq mi,3276/km2,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108...,
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,11900/sq mi,4600/km2,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W...,
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,1651.1 km2,3613/sq mi,1395/km2,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W...,
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,1340.6 km2,3120/sq mi,1200/km2,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901...,


#### - Since the '2016 Land Area' and '2016 Population density' both have two columns representing the same data but in different units, I'll leave only one unit here

In [9]:
# Drop the unneed columns
df2=df1.drop([6,8,11],axis=1)
df2.head(5)

,0,1,2,3,4,5,7,9,10
0,1,New York City,New York,8398748,8175133,+2.74%,780.9 km2,10933/km2,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W...
1,2,Los Angeles,California,3990456,3792621,+5.22%,1213.9 km2,3276/km2,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108...
2,3,Chicago,Illinois,2705994,2695598,+0.39%,588.7 km2,4600/km2,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W...
3,4,Houston,Texas,2325502,2100263,+10.72%,1651.1 km2,1395/km2,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W...
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,1340.6 km2,1200/km2,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901...


#### - Transform columns with numerical value to number data type

In [10]:
df3 = df2.iloc[:,:5].applymap(lambda x: pd.to_numeric(x,errors='ignore'))
df3.head(5)

,0,1,2,3,4
0,1,New York City,New York,8398748,8175133
1,2,Los Angeles,California,3990456,3792621
2,3,Chicago,Illinois,2705994,2695598
3,4,Houston,Texas,2325502,2100263
4,5,Phoenix,Arizona,1660272,1445632


#### - Create a function to extract numerical value from each cell

In [11]:
# Return the numerical value from an input string
def extractNumber(string):
    list_number = re.findall('-?\d+\.?\d*', str(string).replace('−','-'))
    if(len(list_number)==0):
        list_number.append('NA')
    return pd.to_numeric(list_number[0],errors='coerce')

#### - Apply extractNumber function across all cells in columns 'Change', '2016 Land Area', and '2016 Population density'

In [12]:
df4 = df2.iloc[:,5:8].applymap(extractNumber)
df4.head(5)

,5,7,9
0,2.74,780.9,10933
1,5.22,1213.9,3276
2,0.39,588.7,4600
3,10.72,1651.1,1395
4,14.85,1340.6,1200


#### - Create a function to extract Latitude and Longitude from the 'Location' column

In [13]:
# Handle Location info: latitude and longitude
def extractGeoLocation(string):
    lat_long = string.replace('−','-').split('/')[2]
    lat_long_clean = re.findall('-?\d+\.?\d*', lat_long)[:2]
    return lat_long_clean[0]+','+lat_long_clean[1]

In [14]:
dfLocation = df2[10].apply(extractGeoLocation)
dfLocation.head(5)

0     40.6635,-73.9387
1    34.0194,-118.4108
2     41.8376,-87.6818
3     29.7866,-95.3909
4    33.5722,-112.0901
Name: 10, dtype: object

In [15]:
# Split the location column into latitude and longitude
dfLat_Long = dfLocation.str.split(',', expand=True)
dfLat_Long.head(5)

,0,1
0,40.6635,-73.9387
1,34.0194,-118.4108
2,41.8376,-87.6818
3,29.7866,-95.3909
4,33.5722,-112.0901


#### - Concatenate all the clean data table into finalized table

In [16]:
dfAll = pd.concat([df3,df4, dfLat_Long], axis=1)
dfAll.head(5)

,0,1,2,3,4,5,7,9,0,1
0,1,New York City,New York,8398748,8175133,2.74,780.9,10933,40.6635,-73.9387
1,2,Los Angeles,California,3990456,3792621,5.22,1213.9,3276,34.0194,-118.4108
2,3,Chicago,Illinois,2705994,2695598,0.39,588.7,4600,41.8376,-87.6818
3,4,Houston,Texas,2325502,2100263,10.72,1651.1,1395,29.7866,-95.3909
4,5,Phoenix,Arizona,1660272,1445632,14.85,1340.6,1200,33.5722,-112.0901


#### - Clean and modify the headers name for the table

In [17]:
# Locate the column name row
columnNames = rows[0].find_all('th')

# Create a list to store the column names
list_columnNames = []

# Remove the HTML tag in each column name
for columnName in columnNames:
    str_columnName = str(columnName)
    clean = re.compile('<br/>')
    clean1 = re.sub(clean, ' ',str_columnName)
    clean = re.compile('<.*?>|\[.*\]|\\n')
    clean2 = (re.sub(clean, '',clean1.replace(' ','_')))
    list_columnNames.append(clean2)

list_columnNames

['2018_rank',
 'City',
 'State',
 '2018_estimate',
 '2010_Census',
 'Change',
 '2016_land_area',
 '2016_population_density',
 'Location']

In [18]:
# Remove the location header, and add latitude and longitude
list_columnNames1 = list_columnNames[:8]
list_columnNames1.append('Latitude')
list_columnNames1.append('Longitude')
list_columnNames1

['2018_rank',
 'City',
 'State',
 '2018_estimate',
 '2010_Census',
 'Change',
 '2016_land_area',
 '2016_population_density',
 'Latitude',
 'Longitude']

#### - Rename the final data table with correct column names

In [19]:
dfAll.columns = list_columnNames1
dfAll.head(5)

,2018_rank,City,State,2018_estimate,2010_Census,Change,2016_land_area,2016_population_density,Latitude,Longitude
0,1,New York City,New York,8398748,8175133,2.74,780.9,10933,40.6635,-73.9387
1,2,Los Angeles,California,3990456,3792621,5.22,1213.9,3276,34.0194,-118.4108
2,3,Chicago,Illinois,2705994,2695598,0.39,588.7,4600,41.8376,-87.6818
3,4,Houston,Texas,2325502,2100263,10.72,1651.1,1395,29.7866,-95.3909
4,5,Phoenix,Arizona,1660272,1445632,14.85,1340.6,1200,33.5722,-112.0901


In [20]:
# Column names formation and add in units for 'Change', '2016 Land Area', and '2016 Population density'
dfFinal = dfAll.rename(columns={'2018_rank':'Rank_2018',
                        '2018_estimate':'Est_Census_2018',
                        '2010_Census':'Census_2010',
                        '2016_land_area':'Landarea_2016', 
                        '2016_population_density':'Pop_Density_2016'})

dfFinal.head(5)

,Rank_2018,City,State,Est_Census_2018,Census_2010,Change,Landarea_2016,Pop_Density_2016,Latitude,Longitude
0,1,New York City,New York,8398748,8175133,2.74,780.9,10933,40.6635,-73.9387
1,2,Los Angeles,California,3990456,3792621,5.22,1213.9,3276,34.0194,-118.4108
2,3,Chicago,Illinois,2705994,2695598,0.39,588.7,4600,41.8376,-87.6818
3,4,Houston,Texas,2325502,2100263,10.72,1651.1,1395,29.7866,-95.3909
4,5,Phoenix,Arizona,1660272,1445632,14.85,1340.6,1200,33.5722,-112.0901


## Part 2 - Add additional data from each individual city's wikipedia page

#### Since the inital table is related to population, I'll add more population data for each city

#### - From the initial BeautifulSoup object, extract URLs for each city's wikipedia page

In [21]:
# Create a list to store cities's wiki URL
list_cityURLs = []

# Match the patter to extract URL directory
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    pat1 = re.compile('/wiki/[^"]+')
    result = re.search(pat1, str_cells)
    if(result):
        list_cityURLs.append(result.group(0))
    else:
        # if no patter match, set as NaN
        list_cityURLs.append(None)

list_cityURLs

[None,
 '/wiki/New_York_City',
 '/wiki/Los_Angeles',
 '/wiki/Chicago',
 '/wiki/Houston',
 '/wiki/Phoenix,_Arizona',
 '/wiki/Philadelphia',
 '/wiki/San_Antonio',
 '/wiki/San_Diego',
 '/wiki/Dallas',
 '/wiki/San_Jose,_California',
 '/wiki/Austin,_Texas',
 '/wiki/Jacksonville,_Florida',
 '/wiki/Fort_Worth,_Texas',
 '/wiki/Columbus,_Ohio',
 '/wiki/San_Francisco',
 '/wiki/Charlotte,_North_Carolina',
 '/wiki/Indianapolis',
 '/wiki/Seattle',
 '/wiki/Denver',
 '/wiki/Washington,_D.C.',
 '/wiki/Boston',
 '/wiki/El_Paso,_Texas',
 '/wiki/Detroit',
 '/wiki/Nashville,_Tennessee',
 '/wiki/Portland,_Oregon',
 '/wiki/Memphis,_Tennessee',
 '/wiki/Oklahoma_City',
 '/wiki/Las_Vegas',
 '/wiki/Louisville,_Kentucky',
 '/wiki/Baltimore',
 '/wiki/Milwaukee',
 '/wiki/Albuquerque,_New_Mexico',
 '/wiki/Tucson,_Arizona',
 '/wiki/Fresno,_California',
 '/wiki/Mesa,_Arizona',
 '/wiki/Sacramento,_California',
 '/wiki/Atlanta',
 '/wiki/Kansas_City,_Missouri',
 '/wiki/Colorado_Springs,_Colorado',
 '/wiki/Miami',
 '/wik

#### - Create a function to connect with each URL and parse the HTML into BeautifulSoup objects

In [22]:
def getCitySoup(url):
    if url is None:
        return None
    # Set the URL to scrape from
    fullUrl = 'https://en.wikipedia.org'+url
    # Connect to the URL
    response = requests.get(fullUrl)
    # Check if the connect is successful
    if response.status_code != 200:
        print('Fail to connect')
        return None
    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

#### - Create another function to get the Historical population data and return in a form of dictionary

In [23]:
def getPopulationTableDate(soup):
    if soup is None:
        return {}
    popTable = soup.find_all('table', attrs={"class": "toccolours"})
    if(len(popTable))==0:
        return {}
    popRows = popTable[0].find_all('tr')
    year_pop_dict = {}
    for popRow in popRows[2:-1]:
        str_popRow = str(popRow)
        pat = re.compile('>.*?<')
        list_cells = re.findall('>.*?<', str_popRow.replace(',',''))
        pat1 = re.compile('\d+\.?\d*')
        list_numbers = re.findall('\d+\.?\d*', str(list_cells))
        year_pop_dict[list_numbers[0]] = list_numbers[1]
    return year_pop_dict

#### - Start extracting data from each individual city's wikipedia page

In [24]:
year_pop_dict_List = []

for url in progressbar.progressbar(list_cityURLs):
    soup = getCitySoup(url)
    year_pop_dict = getPopulationTableDate(soup)
    year_pop_dict_List.append(year_pop_dict)    

100% (315 of 315) |######################| Elapsed Time: 0:03:06 Time:  0:03:06


#### - Concatenate all the population data into a dataframe for further format

In [25]:
df_year_pop = pd.concat([pd.Series(d) for d in year_pop_dict_List], axis=1, sort=True).T
df_year_pop.head(10)

,1683,1698,1712,1722,1723,1731,1737,1746,1756,1757,1760,1761,1765,1769,1770,1771,1785,1788,1790,1796,1797,1800,1810,1820,1823,1830,1840,1848,1849,1850,1852,1860,1870,1880,1890,1895,1900,1905,1910,1915,1920,1925,1930,1935,1940,1945,1950,1960,1970,1980,1990,2000,2010,2017,2018,2019
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,4937,5840,NaN,7248,NaN,10664,11717,13046,NaN,NaN,NaN,NaN,NaN,NaN,21863,NaN,NaN,49401,NaN,NaN,79216,119734,152056,NaN,242278,391114,NaN,NaN,696115,NaN,1174779,1478103,1911698,2507414,NaN,3437202,NaN,4766883,NaN,5620048,NaN,6930446,NaN,7454995,NaN,7891957,7781984,7894862,7071639,7322564,8008278,8175133,NaN,8398748,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610,NaN,4385,5728,11183,50395,NaN,102479,NaN,319198,NaN,576673,NaN,1238048,NaN,1504277,NaN,1970358,2479015,2811801,2968528,3485398,3694820,3792621,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4470,NaN,NaN,29963,NaN,112172,298977,503185,1099850,NaN,1698575,NaN,2185283,NaN,2701705,NaN,3376438,NaN,3396808,NaN,3620962,3550404,3366957,3005072,2783726,2896016,2695598,NaN,2705994,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2396,NaN,4845,9382,16513,27557,NaN,44633,NaN,78800,NaN,138276,NaN,292352,NaN,384514,NaN,596163,938219,1232802,1595138,1630553,1953631,2100263,2312717,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,1708,3152,NaN,5544,NaN,11314,NaN,29053,NaN,48118,NaN,65414,NaN,106818,439170,581572,789704,983403,1321045,1445632,NaN,1660272,NaN
6,600,NaN,NaN,NaN,NaN,12000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28522,NaN,NaN,41220,53722,63802,NaN,80462,93665,NaN,NaN,121376,NaN,565529,674022,847170,1046964,NaN,1293697,NaN,1549008,NaN,1823779,NaN,1950961,NaN,1931334,NaN,2071605,2002512,1948609,1688210,1585577,1517550,1526006,NaN,1584138,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3488,NaN,8235,12256,20550,37673,NaN,53321,NaN,96614,NaN,161379,NaN,231542,NaN,253854,NaN,408442,587718,654153,785940,935933,1144646,1327407,1511946,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,NaN,731,2300,2637,16159,NaN,17700,NaN,39578,NaN,74361,NaN,147995,NaN,203341,NaN,334387,573224,696769,875538,1110549,1223400,1307402,1419516,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1073,NaN,698,3000,10358,38069,NaN,42639,NaN,92104,NaN,158976,NaN,269475,NaN,294734,NaN,434462,679684,844401,904078,1006977,1188580,1197816,1341075,NaN,NaN


#### - Since there are lots of Null value, I'll drop the columns which has more than half of the data are null

In [26]:
# A list to store column names that are going to be dropped
list_Column_Drops = []
# A list to store new column names
new_column_names = {}

for column in df_year_pop.columns:
    if df_year_pop[column].isna().sum() > 150:
        list_Column_Drops.append(column)
    else:
        new_column_names[column] = 'Census_'+ column

In [27]:
# Drop the columns that are not needed
df1_year_pop = df_year_pop.drop(list_Column_Drops,axis=1)

# Rename the remaining columns
df_year_pop1 = df1_year_pop.rename(columns = new_column_names)
df_year_pop1.head(5)

,Census_1880,Census_1890,Census_1900,Census_1910,Census_1920,Census_1930,Census_1940,Census_1950,Census_1960,Census_1970,Census_1980,Census_1990,Census_2000,Census_2010,Census_2017
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1911698,2507414,3437202,4766883,5620048,6930446,7454995,7891957,7781984,7894862,7071639,7322564,8008278,8175133,NaN
2,11183,50395,102479,319198,576673,1238048,1504277,1970358,2479015,2811801,2968528,3485398,3694820,3792621,NaN
3,503185,1099850,1698575,2185283,2701705,3376438,3396808,3620962,3550404,3366957,3005072,2783726,2896016,2695598,NaN
4,16513,27557,44633,78800,138276,292352,384514,596163,938219,1232802,1595138,1630553,1953631,2100263,2312717


#### - Removing the header row and reindex the table

In [28]:
df_year_pop2= df_year_pop1.drop([0])
df_year_pop_Final = df_year_pop2.reset_index(drop = True).applymap(lambda x: pd.to_numeric(x,errors='coerce'))
df_year_pop_Final

,Census_1880,Census_1890,Census_1900,Census_1910,Census_1920,Census_1930,Census_1940,Census_1950,Census_1960,Census_1970,Census_1980,Census_1990,Census_2000,Census_2010,Census_2017
0,1911698.0,2507414.0,3437202.0,4766883.0,5620048.0,6930446.0,7454995.0,7891957.0,7781984.0,7894862.0,7071639.0,7322564.0,8008278.0,8175133.0,NaN
1,11183.0,50395.0,102479.0,319198.0,576673.0,1238048.0,1504277.0,1970358.0,2479015.0,2811801.0,2968528.0,3485398.0,3694820.0,3792621.0,NaN
2,503185.0,1099850.0,1698575.0,2185283.0,2701705.0,3376438.0,3396808.0,3620962.0,3550404.0,3366957.0,3005072.0,2783726.0,2896016.0,2695598.0,NaN
3,16513.0,27557.0,44633.0,78800.0,138276.0,292352.0,384514.0,596163.0,938219.0,1232802.0,1595138.0,1630553.0,1953631.0,2100263.0,2312717.0
4,1708.0,3152.0,5544.0,11314.0,29053.0,48118.0,65414.0,106818.0,439170.0,581572.0,789704.0,983403.0,1321045.0,1445632.0,NaN
5,847170.0,1046964.0,1293697.0,1549008.0,1823779.0,1950961.0,1931334.0,2071605.0,2002512.0,1948609.0,1688210.0,1585577.0,1517550.0,1526006.0,NaN
6,20550.0,37673.0,53321.0,96614.0,161379.0,231542.0,253854.0,408442.0,587718.0,654153.0,785940.0,935933.0,1144646.0,1327407.0,1511946.0
7,2637.0,16159.0,17700.0,39578.0,74361.0,147995.0,203341.0,334387.0,573224.0,696769.0,875538.0,1110549.0,1223400.0,1307402.0,1419516.0
8,10358.0,38069.0,42639.0,92104.0,158976.0,269475.0,294734.0,434462.0,679684.0,844401.0,904078.0,1006977.0,1188580.0,1197816.0,1341075.0
9,12567.0,18060.0,21500.0,28946.0,39642.0,57651.0,68457.0,95280.0,204196.0,459913.0,629400.0,782248.0,894943.0,945942.0,NaN


## Part 3 - Combine Part 1's result with Part 2 --> Final result

#### - Before concatenation, remove duplicate column ( 'Census_2010' ) and unneed derivative column ( 'Change' )

In [36]:
df_Final = dfFinal.drop(['Census_2010','Change'],axis=1)
df_Final.head(5)

,Rank_2018,City,State,Est_Census_2018,Landarea_2016,Pop_Density_2016,Latitude,Longitude
0,1,New York City,New York,8398748,780.9,10933,40.6635,-73.9387
1,2,Los Angeles,California,3990456,1213.9,3276,34.0194,-118.4108
2,3,Chicago,Illinois,2705994,588.7,4600,41.8376,-87.6818
3,4,Houston,Texas,2325502,1651.1,1395,29.7866,-95.3909
4,5,Phoenix,Arizona,1660272,1340.6,1200,33.5722,-112.0901


#### - Concatenate two final tables

In [37]:
df_result = pd.concat([df_Final, df_year_pop_Final], axis=1, join_axes=[dfFinal.index])
df_result

,Rank_2018,City,State,Est_Census_2018,Landarea_2016,Pop_Density_2016,Latitude,Longitude,Census_1880,Census_1890,Census_1900,Census_1910,Census_1920,Census_1930,Census_1940,Census_1950,Census_1960,Census_1970,Census_1980,Census_1990,Census_2000,Census_2010,Census_2017
0,1,New York City,New York,8398748,780.9,10933,40.6635,-73.9387,1911698.0,2507414.0,3437202.0,4766883.0,5620048.0,6930446.0,7454995.0,7891957.0,7781984.0,7894862.0,7071639.0,7322564.0,8008278.0,8175133.0,NaN
1,2,Los Angeles,California,3990456,1213.9,3276,34.0194,-118.4108,11183.0,50395.0,102479.0,319198.0,576673.0,1238048.0,1504277.0,1970358.0,2479015.0,2811801.0,2968528.0,3485398.0,3694820.0,3792621.0,NaN
2,3,Chicago,Illinois,2705994,588.7,4600,41.8376,-87.6818,503185.0,1099850.0,1698575.0,2185283.0,2701705.0,3376438.0,3396808.0,3620962.0,3550404.0,3366957.0,3005072.0,2783726.0,2896016.0,2695598.0,NaN
3,4,Houston,Texas,2325502,1651.1,1395,29.7866,-95.3909,16513.0,27557.0,44633.0,78800.0,138276.0,292352.0,384514.0,596163.0,938219.0,1232802.0,1595138.0,1630553.0,1953631.0,2100263.0,2312717.0
4,5,Phoenix,Arizona,1660272,1340.6,1200,33.5722,-112.0901,1708.0,3152.0,5544.0,11314.0,29053.0,48118.0,65414.0,106818.0,439170.0,581572.0,789704.0,983403.0,1321045.0,1445632.0,NaN
5,6,Philadelphia,Pennsylvania,1584138,347.6,4511,40.0094,-75.1333,847170.0,1046964.0,1293697.0,1549008.0,1823779.0,1950961.0,1931334.0,2071605.0,2002512.0,1948609.0,1688210.0,1585577.0,1517550.0,1526006.0,NaN
6,7,San Antonio,Texas,1532233,1194.0,1250,29.4724,-98.5251,20550.0,37673.0,53321.0,96614.0,161379.0,231542.0,253854.0,408442.0,587718.0,654153.0,785940.0,935933.0,1144646.0,1327407.0,1511946.0
7,8,San Diego,California,1425976,842.3,1670,32.8153,-117.1350,2637.0,16159.0,17700.0,39578.0,74361.0,147995.0,203341.0,334387.0,573224.0,696769.0,875538.0,1110549.0,1223400.0,1307402.0,1419516.0
8,9,Dallas,Texas,1345047,882.9,1493,32.7933,-96.7665,10358.0,38069.0,42639.0,92104.0,158976.0,269475.0,294734.0,434462.0,679684.0,844401.0,904078.0,1006977.0,1188580.0,1197816.0,1341075.0
9,10,San Jose,California,1030119,459.7,2231,37.2967,-121.8189,12567.0,18060.0,21500.0,28946.0,39642.0,57651.0,68457.0,95280.0,204196.0,459913.0,629400.0,782248.0,894943.0,945942.0,NaN


In [38]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 23 columns):
Rank_2018           314 non-null int64
City                314 non-null object
State               314 non-null object
Est_Census_2018     314 non-null int64
Landarea_2016       314 non-null float64
Pop_Density_2016    314 non-null int64
Latitude            314 non-null object
Longitude           314 non-null object
Census_1880         189 non-null float64
Census_1890         208 non-null float64
Census_1900         221 non-null float64
Census_1910         242 non-null float64
Census_1920         255 non-null float64
Census_1930         267 non-null float64
Census_1940         268 non-null float64
Census_1950         275 non-null float64
Census_1960         294 non-null float64
Census_1970         301 non-null float64
Census_1980         306 non-null float64
Census_1990         309 non-null float64
Census_2000         309 non-null float64
Census_2010         311 non-null float64
Cen

#### - Creste the csv file in the current directory

In [39]:
df_result.to_csv('result.csv', index=False)